In [3]:
import cv2
import mediapipe as mp
import numpy as np

# EAR 계산 함수 정의
def calculate_ear(eye_points):
    # 세로 거리 계산
    V1 = np.linalg.norm(eye_points[1] - eye_points[5])
    V2 = np.linalg.norm(eye_points[2] - eye_points[4])
    # 가로 거리 계산
    H = np.linalg.norm(eye_points[0] - eye_points[3])
    # EAR 계산
    ear = (V1 + V2) / (2.0 * H)
    return ear

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

cap = cv2.VideoCapture(0)

# EAR 임계값 설정
EAR_THRESHOLD = 0.21
EAR_FRAMES = 3
blink_counter = 0
blink_total = 0

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = face_mesh.process(rgb_frame)

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            h, w, _ = frame.shape
            
            # 왼쪽 눈과 오른쪽 눈의 랜드마크 인덱스
            left_eye_indices = [33, 160, 158, 133, 153, 144]
            right_eye_indices = [362, 385, 387, 263, 373, 380]
            
            left_eye_points = np.array([(face_landmarks.landmark[idx].x * w, face_landmarks.landmark[idx].y * h) for idx in left_eye_indices])
            right_eye_points = np.array([(face_landmarks.landmark[idx].x * w, face_landmarks.landmark[idx].y * h) for idx in right_eye_indices])
            
            left_ear = calculate_ear(left_eye_points)
            right_ear = calculate_ear(right_eye_points)
            
            ear_avg = (left_ear + right_ear) / 2.0
            
            # EAR 값에 따른 깜빡임 감지
            if ear_avg < EAR_THRESHOLD:
                blink_counter += 1
            else:
                if blink_counter >= EAR_FRAMES:
                    blink_total += 1
                blink_counter = 0
                
            # 눈 깜빡임 횟수를 화면에 출력
            cv2.putText(frame, f"Blinks: {blink_total}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), 2)

    cv2.imshow('Eye Blink Detection', frame)
    
    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
